# Hluboké zpětnovazební učení

Dneska se podíváme na hluboké zpětnovazební učení, které kombinuje dva námi už známé přístupy -- zpětnovazební učení a hluboké neuronové sítě. Tato technika umožňuje vytvářet mocné agenty, kteří jsou schopni se trénovat sami z hraní her. Tím se učí, jaké tahy jsou dobré a jak hru vyhrát. Takto se agenti naučili některé hry jako dáma a go hrát na úrovni lidských hráčů, a dokonce je i porazit.

## Zpětnovazební učení

Připomeňme si nejprve zpětnovazební učení, které jsme dělali na druhém cvičení. Jeho cílem je naučit našeho agenta, jak se má co nejlépe chovat v nějakém daném prostředí. Necháme ho vykonávat akce, díky kterým se bude přesouvat z jednoho stavu do druhého a od prostředí za to bude dostávat odměnu, kterou se bude snažit maximalizovat. Zpočátku bude provádět náhodné akce a spíše prozkoumávat prostředí, ale později se bude více snažit najít optimální řešení, čili maximalizovat svůj užitek za své chování. Takto se tedy agent naučí nějakou strategii, jak se co nejlépe v daném prostředí chovat. Nezapomeňme, že jednotlivé stavy, mezi kterými se agent přesouvá, nejsou nezávislé a dají se popsat Bellmannovými rovnicemi.

## Q-učení

Q-učení je konkrétní druh zpětnovazebního učení, kde se agent učí odhadovat odměnu pro každý stav a v něm vykonanou akci. Odměnu dostane agent za provedení té akce v tom daném stavu. V klasickém Q-učení jsou tyto hodnoty uložené v matici Q, která má jako řádky stavy a jako sloupce akce. Agent si po provedení akce v daném stavu postupně updatuje hodnoty na odpovídajících souřadnicích v matici, aby pro příště věděl, jak výhodné je tu akci v tom stavu udělat. Problém je v prostředí s hodně stavy a akcemi, protože pak budeme mít obrovskou matici. To se dá vyřešit pomocí hlubokého Q-učení, kde místo matice Q, budeme používat neuronovou síť.


## Hluboké Q-učení

Dnes se tedy podíváme na agenta s hlubokým Q-učením, které kombinuje klasické Q-učení a neuronové sítě. Místo matice Q agentovi definujeme model neuronové sítě, který bude předpovídat hodnoty funkce Q. Síť bude na vstupu dostávat stav a jako výstup bude vracet pravděpodobnosti toho, jak dobré je danou akci udělat. Trénování sítě bude potom probíhat tak, že se porovná aktuální odměna od prostředí s hodnotou spočtenou z Q a minimalizuje se MSE jejich rozdílu. Příklad je inspirován příkladem [odsud](https://keon.io/deep-q-learning/).

In [ ]:
import random
from collections import deque
import tensorflow as tf
import numpy as np
import gymnasium as gym
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.nn import relu
from tensorflow.keras.optimizers import Adam
from keras.models import model_from_json
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

Implementace třídy agenta je celkem přímočará. Definujeme si síť s daným počtem vstupů a výstupů a nastavíme jí všechny parametry potřebné pro trénování Q-učení. Funkce ```build_model``` vytvoří model neuronové sítě se dvěma skrytými vrstvami,  funkce ```action``` vrací akci. Pokud trénujeme, síť používá epsilon-greedy strategii, pokud jen vyhodnocujeme, vrací nejlepší akci. Funkce ```train``` trénuje samotnou neuronovou síť a updatuje váhy.

Protože by ale update vah po každém kroku byl hrozně pomalý a navíc bychom ho dělali jen pro ten jeden poslední krok, uděláme to chytřeji a budeme trénovat síť pomocí tzv. bufferu. Do bufferu si pomocí funkce ```memorize``` budeme ukládat minulý stav, akci, odměnu, nový stav a příznak done, zda se jedná o cílový stav. Po každém kroku si z bufferu vybereme náhodnou trénovací množinu několika minibatchů, na jejichž základě upravíme váhy sítě tak, že spočítáme odměnu pro danou akci a druhou mocninou rozdílu odměny od prostředí a odměny od sítě spočítáme chybu, která se pak backpropaguje zpátky. Toto za nás udělá metoda ```fit```, jen jí musíme předat stavy a požadované výstupy.

Protože ale samotné trénování modelu trvá dlouho, napíšeme si ještě dvě funkce pro ukládání a načítání natrénovaného modelu, abychom nemuseli model přetrénovávat a mohli si ho rovnou nahrát s dříve předtrénovanými váhami. Funkce ```save_model``` pouze serializuje model a natrénované váhy do formátu json a uloží je. Funkce ```load_model``` nám naopak načte a nastaví agentovi uložený model a váhy rovnou bez trénovaní.

In [ ]:
class DeepQLearningAgent:
    def __init__(self, num_inputs, num_outputs, batch_size = 8, num_batches = 16):
        self.num_inputs = num_inputs # state size
        self.num_outputs = num_outputs # action size
        
        self.batch_size = batch_size
        self.num_batches = num_batches
        
        self.epsilon = 1.0 
        self.epsilon_decay = 0.995      
        self.gamma = 0.95
        
        self.memory = deque(maxlen=2000)
        self.build_model()
        self.epsilon_min = 0.01
                 
    def build_model(self):
        self.model = Sequential([])
        self.model.add(Dense(24, activation=relu, input_dim=self.num_inputs, name='dense_11'))
        self.model.add(Dense(24, activation=relu))
        self.model.add(Dense(self.num_outputs, activation='linear'))
        
        opt = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.model.compile(optimizer=opt, loss='mse')
    
    def action(self, state, train=False):
        if train and np.random.uniform() < self.epsilon:
            return np.random.randint(self.num_outputs)
        else: 
            return np.argmax(self.model.predict(state)[0])

    def train(self):
        for _ in range(self.num_batches):
            
            # samplovani minibatche z pameti
            batch = random.sample(self.memory, self.batch_size)
            states = np.array([s for (s, _, _, _, _) in batch])
            
            next_states = np.array([ns for (_, _, _, ns, _) in batch])
            states = states.reshape((-1, self.num_inputs))
            next_states = next_states.reshape((-1, self.num_inputs))
            
            # predikce odmen za akce
            predicted = self.model.predict(states)
            next_predicted = self.model.predict(next_states)
                           
            # spocteni cilove hodnoty
            for i, (state, action, reward, next_state, done) in enumerate(batch):
                predicted[i][action] = reward
                if not done:
                    predicted[i][action] = reward + self.gamma*np.amax(next_predicted[i])

            self.model.fit(states, predicted, epochs=1, verbose=0)
                           
        # snizeni epsilon pro epsilon-greedy strategii
        if self.epsilon > self.epsilon_min:
            self.epsilon = self.epsilon*self.epsilon_decay
    
    def memorize(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
    def save_model(self):
        model_json = self.model.to_json()
        
        with open('model.json', 'w') as json_file:
            json_file.write(model_json)
            
        self.model.save_weights('model.h5')
    
    def load_model(self): 
        json_file = open('model.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        
        loaded_model = model_from_json(loaded_model_json)
        loaded_model.load_weights('model.h5')
        
        self.model = loaded_model

Když už máme napsaného agenta s hlubokým zpětnovazebním učením, vyzkoušíme si ho na nějakém zajímavém úkolu. Tentokrát si zkusíme prostředí [CartPole-v1](https://gym.openai.com/envs/CartPole-v1/), kde máme tyčku na vozíku, který se pohybuje doprava nebo doleva. Cílem je, aby tyčka nespadla. Odměna +1 je získána za každý časový krok, kdy tyč zůstane vzpřímená. Epizoda končí, když je tyč více než 15 stupňů od svislé osy, nebo když se vozík posune více než 2.4 jednotky od středu.

Podíváme se tedy, jak toto prostředí vypadá a pro zajímavost si vypíšeme prostor pozorování a akcí.

In [ ]:
env = gym.make('CartPole-v1', render_mode='human')

print('observation space:', env.observation_space)
print('action space:', env.action_space)

env.reset()
for _ in range(100):
    env.render()
    env.step(env.action_space.sample()) # take a random action
env.close()

Nyní už přejdeme k samotnému trénování. Stav je daný pozicí a rychlostí vozíku a úhlem a rychlostí tyčky a akce jsou pohyb doleva nebo doprava. Vytvoříme si tedy agenta se 4 vstupy a 2 akcemi. Dále si vytvoříme prostředí a pomocné pole pro logování odměn. Pak už můžeme pustit samotné trénování na 1000 epochách a na závěr natrénovaný model uložit.

In [ ]:
agent = DeepQLearningAgent(4, 2)
env = gym.make('CartPole-v1')
rewards = []


for i in range(1001):
    observation, _ = env.reset()
    observation = np.reshape(observation, newshape=(1, -1))
    done = False
    total_reward = 0
    terminated = False
    time = 0
    while not (done or terminated):
        old_state = observation
        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)

        if terminated or truncated:
            observation, info = env.reset()
        total_reward += reward
        time += 1
        reward = reward if not done else 10 # bonus za uplne vyreseni
        observation = np.reshape(observation, newshape=(1, -1))
        agent.memorize(old_state, action, reward, observation, done)
    
    agent.train()
    
    rewards.append(total_reward)
    if i % 100 == 0:
        print(i, total_reward)
agent.save_model()

open_file = open('rewards.txt', 'wb')
pickle.dump(rewards, open_file)
open_file.close()

Vyzkoušíme, jak dobře náš agent umí problém řešit a zároveň si i zobrazíme animaci. Abychom nemuseli dlouho čekat na natrénování vah, načteme si ty předtrénované.

In [ ]:
agent = DeepQLearningAgent(4, 2)
agent.load_model()

env = gym.make('CartPole-v1', render_mode='human')
print('observation space:', env.observation_space)
print('action space:', env.action_space)

observation, _ = env.reset()
observation = np.reshape(observation, newshape=(1, -1))
done = False
terminated = False
total_reward = 0
time = 0
while not (done or terminated):
    env.render()
    old_state = observation
    action = agent.action(observation, train=False)
    observation, reward, done, terminated, _ = env.step(action)
    observation = np.reshape(observation, newshape=(1, -1))
    total_reward += reward
    time += 1     
print(total_reward)
env.close()

Vykreslíme si výsledný graf odměn. Vidíme, že se celkově výsledky zlepšují, ale občas nám spadnou dolu. To se klidně může stát, protože vykreslujeme skóre z jedné hry a občas nějaká holt nevyjde.

In [ ]:
open_file = open('rewards.txt', 'rb')
rewards = pickle.load(open_file)
open_file.close()

plt.figure(figsize=(12,8))
plt.plot(rewards)
plt.ylabel('Reward')
plt.xlabel('Episode')
plt.show()

## Úkol na cvičení

Vyberte si z [OpenAI gym](https://gym.openai.com/envs/) některý z problémů a zkuste ho vyřešit pomocí hlubokého Q-učení. Doporučuji si vybrat nějaké s diskrétními akcemi, ale můžete si klidně vybrat i akce spojité, jen je bude potřeba diskretizovat. V případě diskretizace je dobré mít těch akcí spíše méně, aby síť nemusela mít moc výstupů a netrénovala se příliš dlouho. 